In [4]:
# !pip install selenium

     ---------------------------------------- 6.5/6.5 MB 5.0 MB/s eta 0:00:00
     -------------------------------------- 384.9/384.9 kB 6.0 MB/s eta 0:00:00
     ---------------------------------------- 58.3/58.3 kB ? eta 0:00:00


In [2]:
# !pip install webdriver-manager

In [1]:
!pip install googletrans==4.0.0rc1

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17411 sha256=b2b483668a4394724a025db4369cfb84d95484f574fe4d9085af7a38a3ad804e
  Stored in directory: c:\users\weon3\appdata\local\pip\cache\wheels\60\b3\27\d8aff3e2d5c2d0d97a117cdf0d5f13cd121e2c2b5fb49b55a0
Successfully built googletrans
  Attempting uninstall: googletrans
    Found existing installation: googletrans 3.1.0a0
    Uninstalling googletrans-3.1.0a0:
      Successfully uninstalled googletrans-3.1.0a0


In [33]:
# 필요한 라이브러리 불러오기
import pandas as pd
import numpy as np
import time
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By # 셀레니움 4.0 이상부터는 element에 접근하기위해 필요하다
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import ElementClickInterceptedException
from googletrans import Translator

chromedriver = './chromedriver.exe'

# 구글 번역 객체 생성
translator = Translator()

# 크롤링 할 경로 설정
url = 'https://www.ratebeer.com/search?tab=beer'

# # 페이지 로드를 위한 시간 2초 부여
time.sleep(2)

# craw 함수 작성
def craw(driver, beer, data, k):
    
    # Url open
    print('url_open... {0} 맥주 데이터를 수집합니다..'.format(beer))
    #chrome driver 불어오기
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)
    
    # 맥주 검색
    time.sleep(2)
    element = driver.find_element(By.XPATH, '//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div/input')
    time.sleep(2)
    element.click()
    time.sleep(2)
    element.send_keys(beer)
    time.sleep(3)
    
    # 맥주 선택
    driver.find_element(By.XPATH, '//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div[2]/a[1]/div/div[2]').click()
    
    time.sleep(3)
    beer_name = driver.find_element(By.CSS_SELECTOR, '.MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.mt-3.MuiTypography-h4').text
    
    error_cnt = 0
    
    # 맥주 리뷰 개수 가져오기
    # while문 사용
    while 1:
        try: 
            # 전체 리뷰 개수 수집
            time.sleep(3)
            string = driver.find_element(By.CLASS_NAME, 'MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.MuiTypography-h6').text
            
            # Reviews에 숫자 사이에 천 단위(,)가 있는지에 확인
            # 전체 리뷰개수가 필요한 이유는 Page 수 계산이 필요하기 때문입니다.
            extract = re.compile('[0-9]*,*[0-9]+') # 천 단위(,)를 포함하여 추출
            str_num = extract.findall(string)
            str_num = str_num[0]
            
            print('성공... while문을 탈출합니다.')
            break
        except :
            print('오류 발생.. 다시 시작합니다.')

            error_cnt += 1

            if error_cnt == 5:
                print('연속된 오류로 다음 맥주로 넘어갑니다...')
                return
            
    if ',' in str_num:
        str_num = str_num.split(',')
        str_num = int(str_num[0]+str_num[1])
        num = str_num
    else:
        num = int(str_num)
        
    # Score breakdown element 지정 및 Score breakdown click
    time.sleep(2)
    
    element = driver.find_element(By.XPATH, '//*[@id="root"]/div[2]/div[2]/div/div/div/div[2]/div[1]/div[2]/div/div[2]/div[1]/div[2]')
    time.sleep(2)
    # Score breakdown click
    driver.execute_script("arguments[0].click();", element)
    
    # 첫 번째 Page에서 Show more element 지정 및 Show more click
    time.sleep(2)
    
    buttons = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="root"]/div[2]/div[2]/div/div/div/div//section[2]/div[1]//button')))
    for button in buttons:
        driver.execute_script("arguments[0].click();", button) # 모든 요소를 클릭하라

    # 수집할 페이지 수 계산
    page_num = num // 15 + 1 # 한번에 Review가 기본적으로 15개씩 나온다.
    
    for i in range(page_num):
        print(i+1, '번째 Page입니다.')

        time.sleep(2)
        
        # 각 컬럼별 맥주 데이터 수집 
        # 각 컬럼별로 XPATH 경로를 사용하여 elements 추출
        user_names = []
        user_names_elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//section[1]/div[2]/a/span[1]')))
        for user_names_element in user_names_elements:
            user_name = user_names_element.text
            user_names.append(user_name)

        user_nations = []
        user_nations_elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//div/div/section[1]/div[2]')))
        for user_nations_element in user_nations_elements[:16]:
            user_nation = user_nations_element.text
            user_nations.append(user_nation) # 리스트의 2번째 요소를 저장

        user_ratings = []
        user_ratings_elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//div/div/section[1]/div[2]/div/div[2]/span[1]')))
        for user_rating_elements in user_ratings_elements:
            user_rating = user_rating_elements.text
            user_ratings.append(user_rating)
            
        user_dates = []
        user_date_elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//div/div/section[1]/div[2]/div/div[2]/span[2]')))
        for user_date_element in user_date_elements:
            user_date = user_date_element.text
            user_dates.append(user_date)

        users_aroma = []
        users_aroma_elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//section[2]/div[1]/div[2]/div/div[1]/div[3]')))
        for users_aroma_element in users_aroma_elements:
            user_aroma = users_aroma_element.text
            users_aroma.append(user_aroma)

        users_appearance = []
        users_appearance_elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//section[2]/div[1]/div[2]/div/div[2]/div[3]')))
        for users_appearance_element in users_appearance_elements:
            user_appearance = users_appearance_element.text
            users_appearance.append(user_appearance)

        users_flavor = []
        users_flavor_elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//section[2]/div[1]/div[2]/div/div[3]/div[3]')))
        for users_flavor_element in users_flavor_elements:
            user_flavor = users_flavor_element.text
            users_flavor.append(user_flavor)

        users_mouthfeel = []
        users_mouthfeel_elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//section[2]/div[1]/div[2]/div/div[4]/div[3]')))
        for users_mouthfeel_element in users_mouthfeel_elements:
            user_mouthfeel = users_mouthfeel_element.text
            users_mouthfeel.append(user_mouthfeel)

        users_overall = []
        users_overall_elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//section[2]/div[1]/div[2]/div/div[5]/div[3]')))
        for users_overall_element in users_overall_elements:
            user_overall = users_overall_element.text
            users_overall.append(user_overall)

        users_comments = []
        users_comments_elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//section[2]/div[1]/div[1]/div[1]')))
        for users_comments_element in users_comments_elements:
            user_comments = users_comments_element.text
            users_comments.append(user_comments)

    
        # DataFrame 생성
        tmp = pd.DataFrame(data = zip(user_names, user_nations, user_ratings, user_dates, users_aroma, users_appearance, users_flavor, 
                                      users_mouthfeel, users_overall, users_comments), 
                           columns = ['User_name', 'User_nation', 'Rating', 'Date', 'Aroma', 'Appearance', 
                                              'Flavor', 'Mouthfeel', 'Overall', 'Review'])
        tmp['Beer_name'] = beer_name
        data = pd.concat([data, tmp])
        
        # 다음 페이지로 넘어가기
        # div, span, title 태그 후 속성은 class 외에도 사용 가능
        try :
            element = driver.find_element(By.XPATH, '//button[@title="Next page"]/span[@class="MuiIconButton-label"]')
            time.sleep(3)
            driver.execute_script("arguments[0].click();", element)
            
            time.sleep(2)
            
            # 다음 페이지로 넘어가면서 Show more 버튼을 전부 클릭하기
            buttons = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="root"]/div[2]/div[2]/div/div/div/div//section[2]/div[1]//button')))
            for button in buttons:
                driver.execute_script("arguments[0].click();", button) # 모든 요소를 클릭하라
        except:
            print('마지막 페이지입니다.')
                  
    # 중복 수집 방지
    if num != len(data):
        data = data[:num]

    print('리뷰수 : ', num, '수집된 리뷰수 : ', len(data))
    
    tmp2 = []

    for user_nation_str in data['User_nation'].iloc[:len(data)]:
        tmp2.append(user_nation_str.split('\n'))

    for i in range(0, len(tmp2)):
        data['User_nation'].iloc[i] = tmp2[i][1]
        
    data['Review_kor_tns'] = data['Review'].apply(translator.translate, dest='ko').apply(getattr, args=('text',))

    # 데이터를 csv파일로 저장합니다.
    result = data.to_csv("beer_n_"+str(k)+".csv", encoding='utf-8', index = False)
    

    # 함수 결과 값 반환
    return result

In [34]:
# Beer_data 수집 시작
beer_list = ["Cass Light"]
beer_list = pd.DataFrame(data=beer_list, columns=['검색이름'])

for k in range(len(beer_list)):
    result = craw(chromedriver, beer_list.iloc[k], data, k)

url_open... 검색이름    Cass Light
Name: 0, dtype: object 맥주 데이터를 수집합니다..


C:\Users\weon3\AppData\Local\Temp\ipykernel_8468\1108498605.py:33: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)


성공... while문을 탈출합니다.
1 번째 Page입니다.
2 번째 Page입니다.
3 번째 Page입니다.
리뷰수 :  33 수집된 리뷰수 :  33


In [35]:
beer_df = pd.read_csv('beer_n_0.csv')
beer_df

,User_name,User_nation,Rating,Date,Aroma,Appearance,Flavor,Mouthfeel,Overall,Review,Review_kor_tns,Beer_name
0,Susie33,"🇺🇸Staunton, United States",4.0,"June 16, 2021",-,-,-,-,-,Bubbly gold pour. Thin foam. Hops and yeast..s...,거품이 많은 금 붓기.얇은 거품.홉과 효모 .. 달콤한 밀과 홉 마감 처리 된 달콤...,Cass Light
1,obguthr,"🇺🇸Staunton, United States",2.0,"June 16, 2021",3,3,4,3,7,Tall boy @ Foodmaxx Harrisonburg VA: Rice nose...,"키가 큰 소년 @ FoodMaxx Harrisonburg VA : 쌀 코, 가장자리...",Cass Light
2,Oakes,"🇨🇦Richmond, Canada",0.8,"February 27, 2020",2,1,2,1,2,"Very light, transluscent, watery and appley. ...","매우 가볍고 반전, 물과 Appley.에드.정말 좋지 않습니다.",Cass Light
3,Boxereddy,🇧🇪Belgium,2.5,"April 28, 2019",4,3,5,3,10,"Neus: graan, brood\nUiterlijk: helder gouden b...","코 : 곡물, 빵\n외관 : 넓은 흰색 거품 칼라가있는 맑은 황금 맥주\n맛 : 약...",Cass Light
4,CiderKeep36,"2.7January 25, 2019",2.7,"January 25, 2019",5,3,5,2,12,"Honestly, for a light beer made for mass marke...","솔직히, 대량 시장을 위해 만든 가벼운 맥주의 경우, 같은 종류의 대부분보다 더 낫...",Cass Light
5,crossovert,"🇺🇸Chicago, United States",1.9,"September 27, 2018",4,3,3,3,6,33cl bottle. The marketing for this is trying ...,33Cl 병.이것에 대한 마케팅은 가능한 한 열심히 미국이 되려고 노력하고 있습니다...,Cass Light
6,TBone,"🇫🇮Pori, Finland",1.6,"June 28, 2018",3,2,3,2,6,"Can 500ml (in Jeju). Pale golden colour, small...","500ml (Jeju).창백한 황금색, 작은 흰색 머리.중성 코니 아로마.가벼운 바...",Cass Light
7,AlexRit,🇭🇰Hong Kong,1.2,"May 27, 2018",2,2,2,2,4,"Pale colour. Very light taste, quite watery. T...","옅은 색.매우 가벼운 맛, 꽤 물이 있습니다.전형적인 한국 상업용 라거.맥주 애호가...",Cass Light
8,fredthecat,🇨🇦Canada,1.4,"April 13, 2018",3,1,3,1,6,"industrial smell, piss clear with no head. tas...","산업용 냄새, 머리가없는 소변.맛은 소다수, 단단한 탄산입니다.좋지는 않지만 CAS...",Cass Light
9,jbruner,"🇺🇸Lakewood, United States",2.1,"April 8, 2018",5,3,5,2,6,Pours a clear straw yellow on tap with bubbly ...,맑은 흰색 머리로 맑은 빨대를 붓는 거품이 많은 흰색 머리를 붓습니다.가벼운 부속 ...,Cass Light


# 3월 29일 일지

## 성과 정리
- 현재 리뷰에서 필요한 정보별로 데이터를 모두 가져오는 것은 성공하였으나 다른 유저의 정보를 가져오는 것에서 에러가 발생한다. for문 작동 방식에 문제가 있는 것으로 판단됨.

# 3월 30일 일지

## 성과 정리
- XPATH 경로를 활용하여 'Show more' 버튼을 전부 클릭하게 만드는게 성공하였다.
- 각 컬럼별 구성도 이 코드를 참고하여 모든 페이지를 수집할 수 있게 바꿀 것이다.

# 4월 2일 일지

## 성과 정리
- 모든 페이지를 돌리면서 텍스트를 출력하는데 성공하였다.
- 다만 예상치 못하게 2page부터 Show more을 클릭하지 않는 상황이 발생하였다.
- driver.execute_script를 사용하여 모든 요소를 클릭하는데 성공하였다.

# 4월 9일 일지

## 성과 정리
- 일부 리뷰를 가져오지 않는 문제점을 해소하였다.
- 문제 원인은 국가 경로에서 국가를 입력하지않으면 데이터가 1칸씩 밀리면서 실제 유저와 리뷰 내용이 일치하지않는 심각한 오류가 있었다.
- 이 문제를 해소하기위하여 유저 이름, 국가, 평점, 날짜를 한번에 수집하였고 여기서 국가만 추출하여 User_nation에 다시 삽입하였다.
- 이러면 모든 리뷰를 순서대로 불러올 수 있다.

# 4월 11일 일지

## 성과 정리
- googletrans 라이브러리를 크롤링 데이터에 적용하여 원문과 번역문을 둘 다 얻을 수 있게되었다.
- 한글 언어 모델도 적용해보기 위하여 이런 시도가 필요하였다.